In [1]:
import sys
import os

import pandas as pd
%matplotlib inline
import urllib.request

In [2]:
try:
    import KustoPandas
except:
    sys.path.insert(0, os.path.abspath(os.path.join(os.path.abspath(""), '..')))
    import KustoPandas
from KustoPandas import Wrap

Get a sample dataset and import it as a Pandas DataFrame

In [3]:
def dowload_dataset_if_necessary(url, filename):
    if not os.path.exists(filename):
        urllib.request.urlretrieve (url, filename)
url = "https://projects.fivethirtyeight.com/trump-approval-data/approval_polllist.csv"
filename = "approval_polllist.csv"
dowload_dataset_if_necessary(url, filename)
data = pd.read_csv(filename, parse_dates=["modeldate", "startdate", "enddate"])

wrap it using KustoPandas

In [4]:
w = Wrap(data)

Now we can start to explore it using KustoPandas commands.  

Lets start with something simple and just look at a few lines.

The corresponding kusto command would be 

```
w | take 5
```

Python doesn't support `\`, so instead we use `.`

In [5]:
w.take(5)

,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,...,disapprove,adjusted_approve,adjusted_disapprove,multiversions,tracking,url,poll_id,question_id,createddate,timestamp
0,Donald Trump,All polls,2020-07-23,2017-01-20,2017-01-22,Gallup,B,1500.0,a,0.262323,...,45.0,45.762709,43.573362,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49253,77265,1/23/2017,17:15:30 23 Jul 2020
1,Donald Trump,All polls,2020-07-23,2017-01-20,2017-01-22,Morning Consult,B/C,1992.0,rv,0.680029,...,37.0,45.140403,37.897139,NaN,NaN,http://static.politico.com/9b/13/82a3baf542ae9...,49249,77261,1/23/2017,17:15:30 23 Jul 2020
2,Donald Trump,All polls,2020-07-23,2017-01-21,2017-01-23,Gallup,B,1500.0,a,0.242845,...,46.0,45.762709,44.573362,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49262,77274,1/24/2017,17:15:30 23 Jul 2020
3,Donald Trump,All polls,2020-07-23,2017-01-20,2017-01-24,Ipsos,B-,1632.0,a,0.153481,...,45.2,43.139738,43.865897,NaN,T,http://polling.reuters.com/#poll/CP3_2/,49426,77599,3/1/2017,17:15:30 23 Jul 2020
4,Donald Trump,All polls,2020-07-23,2017-01-22,2017-01-24,Rasmussen Reports/Pulse Opinion Research,C+,1500.0,lv,0.200411,...,43.0,51.572471,44.385622,NaN,T,http://www.rasmussenreports.com/public_content...,49266,77278,1/25/2017,17:15:30 23 Jul 2020


Notice that the output is a nicely formatted table.  That is becuase KustoPandas is just a shallow wrapper around a pandas dataframe.  Pandas is doing the hard work of formatting it nicely in jupyter

Lets explore the data a bit more using the summarize operator.  Here is the Kusto command

```
w | summarize count(), min(startdate), max(startdate), dcount(pollster), AverageDisapproval = avg(disapprove)"
```

The way we execute this in python is that the top level kusto operator (summarize) is a class method. All arguments are passed to the method as a string

In [6]:
w.summarize("count(), min(startdate), max(startdate), dcount(pollster), AverageDisapproval = avg(disapprove)")

,count_,min_startdate,max_startdate,dcount_pollster,AverageDisapproval
0,14151,2017-01-20,2020-07-21,84,53.337543


Similarly we can use the `where` operator to do filtering.  The Kusto query would be

```
w | where grade in ("A", "B")
```

Again (and this is always the case) the top level operator `where` is the class method and the rest of the arguments are passed as a string.

But the output of the above would be too long, so lets chain operators together

```
w | where grade in ("A", "B") | summarize count() by grade, pollster
```

In [7]:
w.where("grade in ('A', 'B')").summarize("count(), avg(disapprove) by grade, pollster")

,grade,pollster,count_,avg_disapprove
0,A,CBS News,41,54.024390
1,A,Suffolk University,30,52.640000
2,A,SurveyUSA,4,51.500000
3,B,American Research Group,109,57.440367
4,B,GQR Research,28,54.273810
5,B,Gallup,840,55.545238
6,B,Public Policy Polling,80,52.950000
7,B,YouGov,4308,52.132916


KustoPandas fully supports arbitrary mathematical expressions, just like kusto

w.extend("NewWeight = exp(weight - 4) * 0.5").summarize("min(NewWeight), max(NewWeight), avg(NewWeight)")

It also supports kusto's nice syntax for binning time values

In [9]:
w.summarize("count() by bin(startdate, 1d)").take(5)

,bin_startdate,count_
0,2017-01-20,9
1,2017-01-21,5
2,2017-01-22,7
3,2017-01-23,12
4,2017-01-24,7


Notice that all the above commands leave `w` unchanged.  In fact that will always be true.  All operations will leave `w` unchanged and return a new instance of the `Wrap` class wrapping a new instance of a Pandas DataFrame.  This is generally the behavior that pandas follows as well, and Pandas makes it easy to do this without duplicating the data inside the dataframe.

If you want to save the output of a calculation, you can do just that

In [11]:
A_rated = w.where("grade == 'A'")
print("number of polls with grade A")
print(A_rated.count())
print("total number of polls")
print(w.count())

number of polls with grade A
   Count
0     75
total number of polls
   Count
0  14151


There are some things that are easier to do in python and Pandas than with kusto, so KustoPandas provides easy access to to the underlying dataframe using `w.df`

For example, we can format the output in the above expression more nicely by accessing the dataframe directly

In [20]:
print("number of polls with grade A: ", A_rated.df.shape[0])
print("total number of polls:        ", w.df.shape[0])

number of polls with grade A:  75
total number of polls:         14151


Of couse the above example is a bit contrived and the info can be written more succinctly using summarize

In [21]:
w.summarize("TotalNumberOfPolls=count(), NumberOfPollsWithGradeA = countif(grade == 'A')")

,TotalNumberOfPolls,NumberOfPollsWithGradeA
0,14151,75
